In [3]:
# ───────────────────────────────────────────────────────────────────────────────
# PREPROCESS CNN-5d
# ───────────────────────────────────────────────────────────────────────────────
import sys, pathlib, joblib, pandas as pd, numpy as np

PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg      # contiene WINDOW, rutas, etc.

# 1) Precios limpitos ──────────────────────────────────────────────────────────
df = (
    pd.read_parquet(cfg.DATA / "raw" / "prices.parquet")
      .ffill()
      .dropna()
)
n_assets = df.shape[1]

# 2) Retornos y features --------------------------------------------------------
ret      = np.log(df / df.shift(1)).dropna()          # r₁d
ret5     = ret.rolling(5).sum()                       # r₅d pasado
vol5     = ret.rolling(5).std()
momentum = (ret5 / (vol5 + 1e-6)).clip(-10, 10)

# 3) Target: retorno 5-días futuro  →  escala “por-día” -------------------------
y_future = (ret5.shift(-5) / 5.0).dropna()            #  r̂ ≈ r_diario futuro

# 4) Desfase de 1 día en features (no mirar el futuro) --------------------------
ret      = ret.shift(1).reindex(y_future.index)
momentum = momentum.shift(1).reindex(y_future.index)

# 5) Ventanas deslizadas  (shape = N,W,n_assets,2) ------------------------------
X, y_out, fechas = [], [], []

for i in range(cfg.WINDOW, len(y_future)):
    win_ret = ret.iloc[i - cfg.WINDOW : i].values      # (W, n_assets)
    win_mom = momentum.iloc[i - cfg.WINDOW : i].values # (W, n_assets)
    bloque  = np.stack([win_ret, win_mom], axis=-1)    # (W, n_assets, 2)

    if not np.isnan(bloque).any():
        X.append(bloque.astype(np.float32))
        y_out.append(y_future.iloc[i].values.astype(np.float32))
        fechas.append(y_future.index[i])

X_arr = np.asarray(X, dtype=np.float32)                # (N, W, n_assets, 2)
y_arr = np.asarray(y_out, dtype=np.float32)            # (N, n_assets)
fechas = pd.to_datetime(fechas)

print("✅ X shape :", X_arr.shape)
print("✅ y shape :", y_arr.shape)
print("✅ Fechas  :", fechas.min().date(), "→", fechas.max().date())



✅ X shape : (4450, 60, 40, 2)
✅ y shape : (4450, 40)
✅ Fechas  : 2012-08-22 → 2025-06-21


In [5]:

# 6. Guardar dataset -----------------------------------------------------------
joblib.dump(
    {"X": X_arr, "y": y_arr, "tickers": df.columns.tolist(), "fechas": fechas},
    cfg.DATA / "processed" / "cnn5d_data.pkl"
)
print("✅ Dataset cnn5d_data.pkl guardado.")


✅ Dataset cnn5d_data.pkl guardado.


In [6]:
# ✅ CORRECCIÓN: Re-guardar con las fechas correctas
# El archivo anterior tenía un error en la variable 'dates'
joblib.dump(
    {"X": X_arr, "y": y_arr, "tickers": df.columns.tolist(), "dates": fechas},
    cfg.DATA / "processed" / "cnn5d_data.pkl"
)
print("✅ Dataset cnn5d_data.pkl corregido y guardado nuevamente.")


✅ Dataset cnn5d_data.pkl corregido y guardado nuevamente.
